In [1]:

"""
This Jupyter notebook provides a Graphical User Interface in order to execute a statistical calculation and plot the result data.
"""
# so that it reloads linked source code files automatically before execution
%load_ext autoreload
%autoreload 2

# Local imports
import Manager
import Plotter_AltitudeProfiles
import Plotter_ColoredDistributions
import Plotter_AltProfComparison
import Data
from scicolorscales import *

# System imports
import os
import glob
import ipywidgets as w
import threading 
import time

import warnings
warnings.filterwarnings(action='once') # warnings.filterwarnings('ignore')


# progress bar 
ProgressBar = w.IntProgress( value=0, min=0, max=100, description='Progress:', bar_style='success', style={'bar_color': 'RoyalBlue'}, orientation='horizontal')
class ProgressBarRefresher(threading.Thread):
    def __init__(self):
        threading.Thread.__init__(self)
    def run(self):
        while True:
            if Data.Progress > 0: 
                ProgressBar.value = Data.Progress
                ProgressBar.layout.visibility = 'visible'
            time.sleep(5)
            
            
######################## Graphical User Interface ########################
def createGUI():
    ## general
    style1 = {'description_width':'170px'}
    style_large = {'description_width':'400px'}
    layout1 = {'width':'850px'}
    layout_small = {'width':'40px'}
    layout_medium = {'width':'180px'}
    layout_large = {'width':'456px'}
    ## the top level visual elements
    MainPanel = w.VBox()    
    MainTab = w.Tab()
    CalcPanel = w.VBox()
    PlotPanel = w.VBox()
    MainPanel.children = [ MainTab ]
    MainTab.children = [ CalcPanel, PlotPanel ]    
    MainTab.set_title(0, 'Calculate')
    MainTab.set_title(1, 'Plot')    
    ## visual elements for calculation
    CalcTypes = list()
    CalcTypes.append( "Ohmic ~ Joule Heating (Ohmic)" )
    CalcTypes.append( "JHminusWindHeat ~ Ohmic minus Wind heating" )
    CalcTypes.append( "SIGMA_PED ~ Pedersen Conductivity" )
    CalcTypes.append( "SIGMA_HAL ~ Hall Conductivity" )
    CalcTypes.append( "EEX_si ~ Electric Field East" )
    CalcTypes.append( "EEY_si ~ Electric Field North" )
    CalcTypes.append( "Convection_heating ~ Convection heating" )
    CalcTypes.append( "Wind_heating ~ Wind heating" )
    TypeOfCalculation_Dropdown = w.Dropdown( options=CalcTypes, description='Calculation: ', style=style1, layout=layout1)
    DistributionNumOfSlots_Textbox = w.Text(value="100", description="Store distribution for each bin with num of slots =", style=style_large, layout=layout_large)
    MLTfrom_box = w.FloatText( value=15, description='MLT from', layout=layout_medium )
    MLTto_box   = w.FloatText( value=39, description='MLT to', layout=layout_medium )
    MLTstep_box = w.FloatText( value=6, description='MLT step', layout=layout_medium )
    ALTfrom_box = w.FloatText( value=95, description='ALT from', layout=layout_medium )
    ALTto_box   = w.FloatText( value=150, description='ALT to', layout=layout_medium )
    ALTstep_box = w.FloatText( value=4, description='ALT step', layout=layout_medium )
    LATfrom_box = w.FloatText( value=71, description='LAT from', layout=layout_medium )
    LATto_box   = w.FloatText( value=78.5, description='LAT to', layout=layout_medium )
    LATstep_box = w.FloatText( value=7.5, description='LAT step', layout=layout_medium )
    KPnum_of_bins_box = w.FloatText( value=3, description='#Kp bins', layout=layout_medium )    
    NetCDFpaths = list()
    NetCDFpaths.append("./TIEGCM_DATA/*/*.nc")
    NetCDFpaths.append("/media/12TB/AllVars/TIEGCM*/*AllVars*.nc")
    NetCDFpath_Dropdown = w.Dropdown( options=NetCDFpaths, description='Input files: ', style=style1, layout=layout1)   #NetCDFpath_Dropdown = w.Dropdown( options=["/media/daedalus/8TB/TIEGCM_DATA/TIEGCM*/*AllVars*.nc"], description='Input files: ', style=style1, layout=layout1)
    TmpFilesPath_Dropdown = w.Dropdown( options=("./tmp_files/", "/media/balukid/STATStmp/",), description='Tmp files path: ', style=style1, layout=layout1)
    Calc_Btn = w.Button(description='Calculate',tooltip="Click here to calculate Statistics",)
    Calc_Btn.style.button_color = 'Gold'
    CalcPanel.children += ( w.HBox( [TypeOfCalculation_Dropdown] ), )
    CalcPanel.children += ( NetCDFpath_Dropdown, )
    CalcPanel.children += ( TmpFilesPath_Dropdown, )
    CalcPanel.children += ( DistributionNumOfSlots_Textbox, )
    CalcPanel.children += ( w.HBox( [MLTfrom_box, MLTto_box, MLTstep_box] ), )
    CalcPanel.children += ( w.HBox( [ALTfrom_box, ALTto_box, ALTstep_box] ), )
    CalcPanel.children += ( w.HBox( [LATfrom_box, LATto_box, LATstep_box] ), )
    CalcPanel.children += ( w.HBox( [KPnum_of_bins_box] ), )
    CalcPanel.children += ( Calc_Btn, )
    ProgressBar.layout.visibility = 'hidden'
    CalcPanel.children += ( ProgressBar, )
    ## visual elements for plotting
    ResultFiles_Dropdown  = w.Dropdown( options=sorted(glob.glob("results/*.nc*")), description='Result files: ', style=style1, layout=layout1)
    ResultFiles2_Dropdown = w.Dropdown( options=[""]+sorted(glob.glob("results/*.nc*")), description='Result files 2: ', style=style1, layout=layout1)
    Plot_Btn = w.Button(description='Plot',tooltip="Click here to plot the results",)
    Plot_Btn.style.button_color = 'YellowGreen'
    PlotPanel.children = [ ResultFiles_Dropdown, ResultFiles2_Dropdown, Plot_Btn ]
    ## Assign event listeners
    def Calc_Btn_Clicked( b ):
        TypeOfCalculation = TypeOfCalculation_Dropdown.value[:TypeOfCalculation_Dropdown.value.rindex('~')].strip()
        Data.setDataParams(MLTfrom_box.value, MLTto_box.value, MLTstep_box.value, LATfrom_box.value, LATto_box.value, LATstep_box.value, ALTfrom_box.value, ALTto_box.value, ALTstep_box.value, KPnum_of_bins_box.value, TypeOfCalculation, int(DistributionNumOfSlots_Textbox.value))
        #print("GUI   ", Data.MLT_min, Data.MLT_max, Data.MLT_duration_of_a_bucket, Data.Alt_min, Data.Alt_max, Data.ALT_distance_of_a_bucket, Data.MagLat_min, Data.MagLat_max, Data.MagLat_degrees_of_a_bucket, Data.num_of_KP_bins, Data.ResultFilename, Data.TypeOfCalculation, Data.KPsequence, Data.ALTsequence, Data.MagLatSequence, Data.MLTsequence )
        if os.path.exists(Data.ResultFilename):
            print("Result file", Data.ResultFilename, "already exists.\nI abort in order to prevent accidental overwrite.")
        else:
            Manager.StartCalculating( NetCDFpath_Dropdown.value, Data.ResultFilename, TypeOfCalculation, TmpFilesPath_Dropdown.value )
    def Plot_Btn_Clicked( b ):
        Buckets2 = None
        if len(ResultFiles2_Dropdown.value) > 0: Buckets2, BinSums2, BinLens2, VariableName2, Units2 = Data.LoadResultsCDF(ResultFiles2_Dropdown.value)
        
        simpleResultsFilename = ResultFiles_Dropdown.value[ ResultFiles_Dropdown.value.replace('\\','/').rindex('/')+1 : ]
        Buckets, BinSums, BinLens, VariableName, Units = Data.LoadResultsCDF(ResultFiles_Dropdown.value)

        Plotter_AltitudeProfiles.plotAltProfiles( VariableName, Buckets, SuperTitle=simpleResultsFilename, ResultsFilename=simpleResultsFilename )
        
        Plotter_ColoredDistributions.plotColoredDistributions( VariableName, Buckets, SuperTitle=simpleResultsFilename )
        
        if VariableName == "Joule Heating":
            Buckets2, BinSums2, BinLens2, VariableName2, Units2 = Data.LoadResultsCDF("results/JouleHeatingMinusNeutralWindHeating__MLT_15_39_6_LAT_68.75_71.25_2.5_ALT_95_150_4_Kp3Bins.nc")
            Plotter_AltProfComparison.plotAltProf_MedianComparison( VariableName, Buckets, "dodgerblue", Buckets2, "dodgerblue", SuperTitle="TIE-GCM Joule heating with and without neutral winds" )
            Plotter_AltProfComparison.plotAltProf_MedianComparison( VariableName, Buckets, "dodgerblue", None, "", SuperTitle="Joule heating in <span style='color:dodgerblue;'>TIE-GCM</span> vs <span style='color:limegreen;'>EISCAT</span>" )
        elif VariableName == "Pedersen Conductivity":
            Plotter_AltProfComparison.plotAltProf_MedianComparison( VariableName, Buckets, "dodgerblue", None, "", SuperTitle="Pedersen conductivity in <span style='color:dodgerblue;'>TIE-GCM</span> vs <span style='color:limegreen;'>EISCAT</span>" )
        else:
            Plotter_AltProfComparison.plotAltProf_MedianComparison( VariableName, Buckets, "dodgerblue", None, "", SuperTitle="TIE-GCM "+VariableName )

    #
    Calc_Btn.on_click( Calc_Btn_Clicked )
    Plot_Btn.on_click( Plot_Btn_Clicked )
    ##
    LATfrom_box.value = 68.75
    LATto_box.value = 71.25
    LATstep_box.value = 2.5
    ##
    return MainPanel

display( createGUI() )

T = ProgressBarRefresher()
T.start()


